In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
from torchvision import datasets, models, transforms
import torch.utils.data as data
import matplotlib.pyplot as plt

import pickle as pkl
import copy
import os

from sklearn.metrics import confusion_matrix
import sys
import utils

In [2]:
all_class = os.listdir("../Images/periodic/all")
print(len(all_class))

remove_classes = ["stone-wall4.o"]
test_classes   = ["B5CC4D9964F342BD.o"]
all_class = [ i for i in all_class if i not in remove_classes ]

train_target_classes = [ i for i in all_class if i not in test_classes ]
test_target_classes =  [ i for i in all_class if i in test_classes ]

print(len(train_target_classes))
print(len(test_target_classes))

55
53
1


In [3]:
train_packs = utils.get_paths(train_target_classes)
test_packs  = utils.get_paths(test_target_classes)
train_transforms = utils.data_transformer_torch_train()
test_transforms  = utils.data_transformer_torch_test()

In [4]:
datasets_train = utils.Img_Dataset(file_list=train_packs[0],transform=train_transforms,labels=train_packs[1],class_labels=train_packs[2])
datasets_test  = utils.Img_Dataset(file_list=test_packs[0] ,transform=test_transforms,labels=test_packs[1],class_labels=test_packs[2])

dataloader_train = torch.utils.data.DataLoader(datasets_train, batch_size=8, shuffle=True,num_workers=8)
dataloader_test  = torch.utils.data.DataLoader(datasets_test , batch_size=8, shuffle=False,num_workers=8)

dataloaders  = {"train":dataloader_train,"val":dataloader_test }
dataset_sizes ={"train":len(datasets_train),"val":len(datasets_test)}

In [5]:
model = models.vgg16(pretrained=True)
model.classifier = nn.Sequential(
        nn.Linear(25088,100),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(100,1) ,nn.Sigmoid())

In [6]:
#criterion = nn.CrossEntropyLoss()
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

In [8]:
model,loss_dict,acc_dict = utils.training_model(dataloaders,dataset_sizes,model,criterion,
                     optimizer,scheduler,num_epochs=5)

Epoch 1/5
----------
---train---
train Loss: 0.5219 ,ACC:0.7373
---val---
val Loss: 0.0888 ,ACC:1.0000
Epoch 2/5
----------
---train---
train Loss: 0.4198 ,ACC:0.8107
---val---
val Loss: 0.4174 ,ACC:0.7000
Epoch 3/5
----------
---train---
train Loss: 0.4031 ,ACC:0.8004
---val---
val Loss: 0.0564 ,ACC:1.0000
Epoch 4/5
----------
---train---
train Loss: 0.3072 ,ACC:0.8672
---val---
val Loss: 0.0513 ,ACC:1.0000
Epoch 5/5
----------
---train---
train Loss: 0.2791 ,ACC:0.8635
---val---
val Loss: 0.0105 ,ACC:1.0000


In [9]:
with open("models/log.pkl","wb") as f:
    pkl.dump([loss_dict,acc_dict],f)

torch.save(model.state_dict(),"models/final_model_wts.pt")